## Helpers

In [20]:
import pandas as pd
from string import punctuation

In [21]:
df = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/dataset_v_1.csv')

In [1]:
# doc = nlp(sent)
# import pytextrank

# nlp.add_pipe("textrank")
# # examine the top-ranked phrases in the document
# for p in doc._.phrases:
#     print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
#     print(p.chunks)
# # https://pypi.org/project/pytextrank/2.0.0/

In [2]:
punctuation_minimal = "!(),-.:;?%"
cyrillic_letters = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ "
allowed_symbols = cyrillic_letters+punctuation_minimal

In [3]:
def clean_text(sent):
    getVals = list(filter(lambda x: x in allowed_symbols, sent))
    result = "".join(getVals).lower()

    return result

In [4]:
def lst_to_str_lemmas(list):
    result = ''
    for bigram in list:
        result += ' '.join([word.lemma_ for word in bigram]) + ', '
    return result.strip(', ')

In [5]:
def lst_to_str(list):
    result = ''
    for bigram in list:
        result += ' '.join([word.text for word in bigram]) + ', '
    return result.strip(', ')

In [6]:
def lst_ofstr_to_str(list):
    # for item in list:
    #     print(item)
    result = ''
    for bigram in list:
        result += bigram[0] + ', '
    return result.strip(', ')

In [7]:
import spacy

nlp = spacy.load("ru_core_news_sm") # load Russian language model

def extract_adj_noun_bigrams(sentence):
    doc = nlp(sentence)
    bigrams = []
    for token in doc:
        if token.pos_ == "ADJ" and token.head.pos_ == "NOUN":
            bigrams.append((token.text, token.head.text))
    return bigrams

In [8]:
flatten = lambda *n: (e for a in n for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

In [9]:
def extract_info(sentence):
    doc = nlp(sentence)
    for token in doc:
        print(token.lemma_, token.pos_, token.dep_)
        print('--------------')
        for child in token.children:
            print(child.lemma_)

In [10]:
# nlp function returns an object with individual token information,
# linguistic features and relationships
from spacy import displacy

def show_dependency(sent):
    doc = nlp(sent)

    print ("{:<15} | {:<8} | {:<15} | {:<20}".format('Token','Relation','Head', 'Children'))
    print ("-" * 70)

    for token in doc:
      print ("{:<15} | {:<8} | {:<15} | {:<20}".format(str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))

    # Use displayCy to visualize the dependency
    displacy.render(doc, style='dep', jupyter=True, options={'distance': 70})

In [25]:
sent = "Попытка создать образ героического защитника Русского мира, по беспределу высылаемого в лапы кровожадных хохлов, говорит лишь о том, что образ этот ДО СИХ ПОР является критически важным для общественного мнения в России."

In [26]:
sent3 = 'я думаю о ласковых и милых котиках и мечтаю их купить, а брат хочет в отпуск '


In [27]:
from random import randrange

def show_dep_example(df, i=None):
    if not(i):
        i = randrange(len(df))

    print(df['text'][i])
    print('*'*20, i, '*'*20)
    print('Nsubj + Verb', ': ', df['core_noun_verb_str'][i])
    print('Appos', ': ', df['appos_str'][i])
    print('Noun+Aj (Amod)', ': ', df['amod_str'][i])
    print('Noun+Noun (Nmod)', ': ', df['nmod_str'][i])
    print('Verb + Obl/Obj', ': ', df['verb_obl_obj_str'][i])

## Noun + verb

In [15]:
def collect_core(sent):
    doc = nlp(clean_text(sent))
    core_list = []
    dicts = [{'token':token, 'pos':token.pos_, 'text':token.text, 'dep':token.dep_, 'lemma':token.lemma_} for token in doc]
    nsubj_list = [item['token'] for item in dicts if item["dep"] == "nsubj"]
    head_verb_list = []
    for nsubj in nsubj_list:
        core_list.append([nsubj, nsubj.head])
        head_verb_list.append(nsubj.head)

    for core_pair in core_list: # check if there are any conj and cross-check that they are not head for other nsubj
        for v_child in core_pair[1].children:
            if (v_child.dep_=='conj') and v_child not in head_verb_list:
                core_list.append([core_pair[0], v_child])


    for core_pair in core_list:

        for v_child in core_pair[1].children:
            advmod = None
            ccomp = None
            xcomp = None
            if v_child.dep_=='advmod':
                advmod = v_child
            elif v_child.dep_=='ccomp':
                ccomp = v_child
            elif v_child.dep_=='xcomp':
                xcomp = v_child

            if any([advmod,ccomp,xcomp]):
                core_pair[1] = [advmod, core_pair[1], ccomp, xcomp]
                core_pair[1] = [item for item in core_pair[1] if item!=None]
                core_pair[1] = list(flatten(core_pair[1]))


    core_list = [list(flatten(item)) for item in core_list]

    return core_list

## Verb + noun

In [126]:
def exclude_empty_vals(list):
    result = [item for item in list if item!=None]
    return result

In [127]:
def collect_verb_obl_obj(sentence):
    doc = nlp(clean_text(sentence))
    result = []
    dicts = [{'token':token, 'pos':token.pos_, 'text':token.text, 'dep':token.dep_, 'lemma':token.lemma_} for token in doc]
    verbs = [item['token'] for item in dicts if item["pos"] == "VERB"]
    for verb in verbs:
        advmod = None
        for v_ch in verb.children:
            if (v_ch.dep_=='advmod') and (sum(1 for _ in v_ch.children)==0):
                advmod = v_ch
                # print(advmod, verb)
                # result.append([advmod, verb])

        for v_child in verb.children:

            # find obl and case
            if v_child.dep_=='obl':
                case = None
                for g_child in v_child.children:
                    if g_child.dep_=='case':
                        case = g_child
                if case:
                    result.append(exclude_empty_vals([advmod, verb, case, v_child]))
                else:
                    result.append(exclude_empty_vals([advmod, verb, v_child]))

            if v_child.dep_=='obj':
                case = None
                for g_child in v_child.children:
                    if g_child.dep_=='case':
                        case = g_child
                if case:
                    result.append(exclude_empty_vals([advmod, verb, case, v_child]))
                else:
                    result.append(exclude_empty_vals([advmod, verb, v_child]))

            # find obj
            if (v_child.dep_=='xcomp') or (v_child.dep_=='ccomp'):
                for grand_child in v_child.children:
                    obj = None
                    if grand_child.dep_=='obj':
                        obj = grand_child
                        for g_child in v_child.children:
                            if g_child.dep_=='case':
                                case = g_child
                        if obj:
                            result.append(exclude_empty_vals([advmod, verb, obj, v_child]))
                        else:
                            result.append(exclude_empty_vals([advmod, verb, v_child]))


    return result


In [128]:
def collect_verb_obl_obj2(sentence):
    doc = nlp(clean_text(sentence))
    result = []
    dicts = [{'token':token, 'pos':token.pos_, 'text':token.text, 'dep':token.dep_, 'lemma':token.lemma_} for token in doc]
    verbs = [item['token'] for item in dicts if item["pos"] == "VERB"]
    for verb in verbs:

        for v_child in verb.children:

            advmod = None
            obj = None
            obl = None
            if v_child.dep_=='obj':
                obj = v_child
            elif v_child.dep_=='obl':
                obl = v_child
            elif v_child.dep_=='advmod':
                advmod = v_child

            if any([advmod,obj,obl]):
                lst = [advmod, verb, obj, obl]
                lst = [item for item in lst if item!=None]
                lst = list(flatten(lst))

                result.append(lst)

    return result


In [130]:
collect_verb_obl_obj(df['text'][335])

[[не, имеем, права],
 [показывать, слабину],
 [пытаются, руку, откусить],
 [откусить, руку],
 [появляется, у, них]]

In [141]:
show_dependency(df['text'][335])

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
✔               | nsubj    | имеем           | [️Мы]               
️Мы             | appos    | ✔               | []                  
ни              | advmod   | имеем           | [в, коем, случае]   
в               | fixed    | ни              | []                  
коем            | fixed    | ни              | []                  
случае          | fixed    | ни              | []                  
не              | advmod   | имеем           | []                  
имеем           | ROOT     | имеем           | [✔, ни, не, права, !]
права           | obj      | имеем           | [показывать]        
показывать      | nmod     | права           | [слабину, террористам]
слабину         | obj      | показывать      | []                  
террористам     | iobj     | показывать      | [Хохлостана]        
из              | case     | Хохлостана   

In [142]:
df['text'][335]

'✔️Мы ни в коем случае не имеем права показывать слабину террористам из Хохлостана! Даёшь им палец, а они пытаются откусить всю руку! Только у них появляется повод для так называемой «перемоги» - сразу же случается очередной жирный всплеск русофобской активности.'

In [143]:
collect_core(df['text'][335])

[[мы, не, ни, имеем],
 [они, пытаются, откусить],
 [повод, появляется],
 [всплеск, сразу, -, случается]]

## Appos

In [144]:
sent5 = 'Карл Четвертый, римский император и чешский король, царствовал в XIV веке'

In [145]:
sent4 = 'я думаю о ласковых и милых котиках и мечтаю про них, а брат хочет в отпуск '

In [146]:
show_dependency(sent5)

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Карл            | nsubj    | царствовал      | [Четвертый, император, король, ,]
Четвертый       | appos    | Карл            | []                  
,               | punct    | император       | []                  
римский         | amod     | император       | []                  
император       | conj     | Карл            | [,, римский]        
и               | cc       | король          | []                  
чешский         | amod     | король          | []                  
король          | conj     | Карл            | [и, чешский]        
,               | punct    | Карл            | []                  
царствовал      | ROOT     | царствовал      | [Карл, веке]        
в               | case     | веке            | []                  
XIV             | amod     | веке            | []                  
веке            | obl      | цар

![Screenshot_1.png](pics/Screenshot_1.png)

In [147]:
sent6 = 'Пришел Михаил , мой брат и кузен Давида .'

In [148]:
show_dependency(sent6)

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Пришел          | ROOT     | Пришел          | [Михаил, .]         
Михаил          | nsubj    | Пришел          | [брат, кузен]       
,               | punct    | брат            | []                  
мой             | det      | брат            | []                  
брат            | conj     | Михаил          | [,, мой]            
и               | cc       | кузен           | []                  
кузен           | conj     | Михаил          | [и, Давида]         
Давида          | nmod     | кузен           | []                  
.               | punct    | Пришел          | []                  


In [149]:
def collect_appos(sent):
    doc = nlp(clean_text(sent))
    dicts = [{'token':token, 'pos':token.pos_, 'text':token.text, 'dep':token.dep_, 'lemma':token.lemma_} for token in doc]
    appos_list = [item['token'] for item in dicts if item["dep"] == "appos"]
    result = []

    for appos in appos_list:
        flat_name = None
        for child in appos.children:
            if child.dep_=='flat:name':
                flat_name = child

        if flat_name:
            result.append([appos.head, appos, flat_name])
        else:
            result.append([appos.head, appos])

    return result

In [150]:
show_dependency('Владимир Путин , российский президент')

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Владимир        | ROOT     | Владимир        | [Путин, президент]  
Путин           | appos    | Владимир        | []                  
,               | punct    | президент       | []                  
российский      | amod     | президент       | []                  
президент       | appos    | Владимир        | [,, российский]     


In [151]:
collect_appos('Владимир Путин , российский президент')

[[владимир, президент]]

## Noun + adj

In [152]:
def collect_amod(sent):
    doc = nlp(clean_text(sent))
    nouns_phrases = []
    dicts = [{'token':token, 'pos':token.pos_, 'text':token.text, 'dep':token.dep_, 'lemma':token.lemma_} for token in doc]
    nouns = [item['token'] for item in dicts if item["pos"] == "NOUN"]

    for noun in nouns:
        for child in noun.children:
            if child.dep_=='amod':
               nouns_phrases.append([child, noun])

    return nouns_phrases

## noun + noun

In [153]:
def collect_nmod(sent):
    doc = nlp(clean_text(sent))
    nouns_phrases = []
    dicts = [{'token':token, 'pos':token.pos_, 'text':token.text, 'dep':token.dep_, 'lemma':token.lemma_} for token in doc]
    nouns_list = [item['token'] for item in dicts if item["pos"] == "NOUN"]

    for noun in nouns_list:
        for child in noun.children:
            if child.dep_=='nmod':

                case = None
                for g_child in child.children:
                    if g_child.dep_=='case':
                        case = g_child

                n_case = None
                for child_ in noun.children:
                    if child.dep_=='case':
                        n_case = child_

                if case and n_case:
                    nouns_phrases.append([n_case, noun, g_child, child])
                elif case and (not(n_case)) :
                    nouns_phrases.append([noun, g_child, child])
                elif (not(case)) and n_case:
                    nouns_phrases.append([n_case, noun, child])
                elif (not(case and n_case)):
                    nouns_phrases.append([noun, child])

    return nouns_phrases

## compound noun

In [154]:
def collect_comp(sent):
    doc = nlp(clean_text(sent))
    nouns_phrases = []
    dicts = [{'token':token, 'pos':token.pos_, 'text':token.text, 'dep':token.dep_, 'lemma':token.lemma_} for token in doc]
    nouns_list = [item['token'] for item in dicts if item["pos"] == "NOUN"]

    for noun in nouns_list:
        for child in noun.children:
            if child.dep_=='compound':
                nouns_phrases.append([noun, child])

    return nouns_phrases

In [155]:
collect_comp('Олег пошел в бизнес центр, типичное поведение в двадцать первом веке')

[]

In [156]:
show_dependency('Олег пошел в бизнес центр, типичное поведение в двадцать первом веке')

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Олег            | nsubj    | пошел           | []                  
пошел           | ROOT     | пошел           | [Олег, бизнес, центр, веке]
в               | case     | бизнес          | []                  
бизнес          | obl      | пошел           | [в]                 
центр           | obj      | пошел           | [поведение]         
,               | punct    | поведение       | []                  
типичное        | amod     | поведение       | []                  
поведение       | conj     | центр           | [,, типичное]       
в               | case     | веке            | []                  
двадцать        | advmod   | первом          | []                  
первом          | amod     | веке            | [двадцать]          
веке            | obl      | пошел           | [в, первом]         


In [157]:
show_dependency('Это будет стоить пятьдесят пять тысяч рублей')

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Это             | nsubj    | стоить          | []                  
будет           | aux      | стоить          | []                  
стоить          | ROOT     | стоить          | [Это, будет, рублей]
пятьдесят       | nummod:gov | тысяч           | []                  
пять            | nummod:gov | тысяч           | []                  
тысяч           | nummod   | рублей          | [пятьдесят, пять]   
рублей          | obj      | стоить          | [тысяч]             


## Working with df

In [158]:
import pandas as pd

In [132]:
df = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/dataset_v_1.csv')

In [133]:
%%time
df['core_noun_verb'] = df['text'].apply(lambda x: collect_core(x))

CPU times: user 28.5 s, sys: 342 ms, total: 28.9 s
Wall time: 29.4 s


In [134]:
%%time
df['verb_obl_obj'] = df['text'].apply(lambda x: collect_verb_obl_obj(x))

CPU times: user 27.4 s, sys: 296 ms, total: 27.7 s
Wall time: 28 s


In [159]:
%%time
df['appos'] = df['text'].apply(lambda x: collect_appos(x))

CPU times: user 27.2 s, sys: 209 ms, total: 27.4 s
Wall time: 27.5 s


In [160]:
%%time
df['amod'] = df['text'].apply(lambda x: collect_amod(x))

CPU times: user 27.1 s, sys: 168 ms, total: 27.2 s
Wall time: 27.3 s


In [161]:
%%time
df['nmod'] = df['text'].apply(lambda x: collect_nmod(x))

CPU times: user 27.3 s, sys: 198 ms, total: 27.5 s
Wall time: 27.6 s


In [162]:
%%time
df['comp'] = df['text'].apply(lambda x: collect_comp(x))

CPU times: user 27.5 s, sys: 254 ms, total: 27.7 s
Wall time: 27.8 s


In [163]:
df['core_noun_verb_str'] = df['core_noun_verb'].apply(lambda x: lst_to_str(x))

In [165]:
df['verb_obl_obj_str'] = df['verb_obl_obj'].apply(lambda x: lst_to_str(x))

In [166]:
df['appos_str'] = df['appos'].apply(lambda x: lst_to_str(x))

In [167]:
df['amod_str'] = df['amod'].apply(lambda x: lst_to_str(x))

In [168]:
df['nmod_str'] = df['nmod'].apply(lambda x: lst_to_str(x))

In [169]:
df

,text,core_noun_verb,verb_obl_obj,core_noun_verb_str,appos,amod,nmod,comp,verb_obl_obj_str,appos_str,amod_str,nmod_str
0,Вагнер отправляет к бандере и прочим шайтанам ...,"[[вагнер, отправляет]]","[[отправляет, к, бандере], [отправляет, бригады]]",вагнер отправляет,"[[батальон, джохара, дудаева]]","[[прочим, шайтанам], [следующие, бригады], [ша...","[[бандере, бахмута, районе], [районе, бахмута]...",[],"отправляет к бандере, отправляет бригады",батальон джохара дудаева,"прочим шайтанам, следующие бригады, шайтанский...","бандере бахмута районе, районе бахмута, бригад..."
1,Тут понимаешь всё про метафизику этой войны. «...,"[[укропы, вышли], [они, вышли]]","[[тут, понимаешь, всё], [тут, понимаешь, про, ...","укропы вышли, они вышли",[],[],"[[метафизику, войны], [укропы, запада, покрово...",[],"тут понимаешь всё, тут понимаешь про метафизик...",,,"метафизику войны, укропы запада покровом, покр..."
2,"Наш источник сообщает, что остатки военной ави...","[[источник, сообщает, переброшена]]","[[переброшена, на, аэродром]]",источник сообщает переброшена,"[[аэродром, рог]]","[[военной, авиации], [долгинцево(кривой, рог]]","[[остатки, авиации], [остатки, укрорейха]]",[],переброшена на аэродром,аэродром рог,"военной авиации, долгинцево(кривой рог","остатки авиации, остатки укрорейха"
3,Раньше укронацисты строили песчаные замки — но...,"[[укронацисты, раньше, строили], [укронацисты,...","[[раньше, строили, замки]]","укронацисты раньше строили, укронацисты теперь...",[],"[[песчаные, замки]]",[],[],раньше строили замки,,песчаные замки,
4,Именно эти регионы являются настоящим рассадни...,"[[регионы, являются, рассадником], [регионы, и...","[[имеют, для, укронациков], [имеют, смысл]]","регионы являются рассадником, регионы имеют",[],"[[настоящим, рассадником], [сакральный, смысл]...","[[рассадником, укронацизма]]",[],"имеют для укронациков, имеют смысл",,"настоящим рассадником, сакральный смысл, идеол...",рассадником укронацизма
...,...,...,...,...,...,...,...,...,...,...,...,...
3638,Стоимость очередной подачки хохлам - $675 млн.,[],[],,[],"[[очередной, подачки], [-, млн]]","[[стоимость, подачки], [подачки, млн]]",[],,,"очередной подачки, - млн","стоимость подачки, подачки млн"
3639,Хохлячие каналы ссылаясь на некую разведку соо...,"[[каналы, сообщают], [вс, готовы, подорвать], ...","[[ссылаясь, на, разведку], [сообщают, о, том],...","каналы сообщают, вс готовы подорвать, зеленски...",[],"[[хохлячие, каналы], [каховской, гэс], [масшта...","[[каналы, ссылаясь], [дамбу, гэс], [шумок, тре...",[],"ссылаясь на разведку, сообщают о том, подорват...",,"хохлячие каналы, каховской гэс, масштабной тре...","каналы ссылаясь, дамбу гэс, шумок тревоги"
3640,"Обсуждают недоукраинца Невзорова, Анну Нетребк...",[],"[[обсуждают, недоукраинца]]",,[],[],[],[],обсуждают недоукраинца,,,
3641,И вот приходит коллективный Зусько и буквально...,"[[зусько, вот, приходит], [зусько, буквально, ...","[[буквально, начинает, хохлов, заваливать], [з...","зусько вот приходит, зусько буквально начинает...",[],"[[коллективный, зусько], [ровном, месте]]","[[трупами, мобилизованных]]",[],"буквально начинает хохлов заваливать, завалива...",,"коллективный зусько, ровном месте",трупами мобилизованных


In [176]:
show_dep_example(df)

🗣На шоу «вечерний Зеленский» в своем выступлении президенту Украины пришлось выбирать между плохими и ужасными новостями  ▪️Он «опустил» сдачу населенных пунктов и тяжелое положение ВСУшников под Артемовском.
******************** 2263 ********************
Nsubj + Verb :  он опустил
Appos :  
Noun+Aj (Amod) :  плохими новостями, населенных пунктов, тяжелое положение
Noun+Noun (Nmod) :  шоу вечерний, президенту украины, сдачу пунктов, положение всушников, положение под артемовском
Verb + Obl/Obj :  пришлось на шоу, пришлось в выступлении, выбирать между новостями, опустил сдачу


In [171]:
show_dep_example(df, 335)

✔️Мы ни в коем случае не имеем права показывать слабину террористам из Хохлостана! Даёшь им палец, а они пытаются откусить всю руку! Только у них появляется повод для так называемой «перемоги» - сразу же случается очередной жирный всплеск русофобской активности.
******************** 335 ********************
Nsubj + Verb :  мы не ни имеем, они пытаются откусить, повод появляется, всплеск сразу - случается
Appos :  
Noun+Aj (Amod) :  очередной всплеск, жирный всплеск, русофобской активности
Noun+Noun (Nmod) :  права показывать, террористам из хохлостана, повод называемой перемоги, всплеск активности
Verb + Obl/Obj :  не имеем права, показывать слабину, пытаются руку откусить, откусить руку, появляется у них


In [172]:
show_dep_example(df, 1082)

На хохляцком ресурсе увидел как шумеры называют Карабахскую войну - "війна в Карабасі".
******************** 1082 ********************
Nsubj + Verb :  шумеры называют
Appos :  
Noun+Aj (Amod) :  хохляцком ресурсе, карабахскую войну
Noun+Noun (Nmod) :  
Verb + Obl/Obj :  увидел на ресурсе, увидел войну называют, увидел - называют, увидел вйна называют, называют войну, называют -, называют вйна, называют в карабас


In [173]:
show_dep_example(df, 152)

Украина не страна и не государство, и не Бандера с автоматом.
******************** 152 ********************
Nsubj + Verb :  украина не страна, украина не государство, украина не бандера
Appos :  
Noun+Aj (Amod) :  
Noun+Noun (Nmod) :  бандера с автоматом
Verb + Obl/Obj :  


## Smth weird

In [940]:
show_dep_example(df, 2405)

Хохлы ударили по Антоновскому мосту в момент эвакуации гражданских.
******************** 2405 ********************
Nsubj + Verb :  
Appos :  
Noun+Noun (Amod) :  Антоновскому мосту
Noun+Noun (Nmod) :  момент эвакуации, эвакуации гражданских
Verb + Obl/Obj :  ударили хохлы, ударили по мосту, ударили в момент


In [938]:
show_dependency(df['text'][2405])

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Хохлы           | obj      | ударили         | []                  
ударили         | ROOT     | ударили         | [Хохлы, мосту, момент, .]
по              | case     | мосту           | []                  
Антоновскому    | amod     | мосту           | []                  
мосту           | obl      | ударили         | [по, Антоновскому]  
в               | case     | момент          | []                  
момент          | obl      | ударили         | [в, эвакуации]      
эвакуации       | nmod     | момент          | [гражданских]       
гражданских     | nmod     | эвакуации       | []                  
.               | punct    | ударили         | []                  


In [963]:
# Пунктуація = nsunbj? як зрозуміти що воно не монолітне?
show_dependency(df['text'][1952])

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Нельзя          | ROOT     | Нельзя          | [отстранять, вице, -, президент, Нельзя, заявил, .]
отстранять      | csubj    | Нельзя          | [сборную, участия]  
всю             | det      | сборную         | []                  
сборную         | obj      | отстранять      | [всю, РФ]           
РФ              | nmod     | сборную         | []                  
от              | case     | участия         | []                  
участия         | obl      | отстранять      | [от, играх]         
в               | case     | играх           | []                  
Олимпийских     | amod     | играх           | []                  
играх           | nmod     | участия         | [в, Олимпийских]    
–               | punct    | вице            | []                  
вице            | nsubj    | Нельзя          | [–]                 
-             

In [966]:
show_dependency(df['text'][1935])

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
9               | obl      | принял          | [октября]           
октября         | flat     | 9               | []                  
Посол           | nsubj    | принял          | [России, Индии, Денис]
России          | nmod     | Посол           | []                  
в               | case     | Индии           | []                  
Индии           | nmod     | Посол           | [в]                 
Денис           | appos    | Посол           | [Алипов]            
Алипов          | flat:name | Денис           | []                  
принял          | ROOT     | принял          | [9, Посол, участие, .]
участие         | obj      | принял          | [фестивале]         
в               | case     | фестивале       | []                  
традиционном    | amod     | фестивале       | []                  
осеннем         | amod     | фестивале  

In [1003]:
show_dependency(df['text'][335])

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
✔               | nsubj    | имеем           | [️Мы]               
️Мы             | appos    | ✔               | []                  
ни              | advmod   | имеем           | [в, коем, случае]   
в               | fixed    | ни              | []                  
коем            | fixed    | ни              | []                  
случае          | fixed    | ни              | []                  
не              | advmod   | имеем           | []                  
имеем           | ROOT     | имеем           | [✔, ни, не, права, !]
права           | obj      | имеем           | [показывать]        
показывать      | nmod     | права           | [слабину, террористам]
слабину         | obj      | показывать      | []                  
террористам     | iobj     | показывать      | [Хохлостана]        
из              | case     | Хохлостана   

In [179]:
clean_text(df['text'][335])

'мы ни в коем случае не имеем права показывать слабину террористам из хохлостана! даёшь им палец, а они пытаются откусить всю руку! только у них появляется повод для так называемой перемоги - сразу же случается очередной жирный всплеск русофобской активности.'

In [1042]:
show_dep_example(df, 335)

✔️Мы ни в коем случае не имеем права показывать слабину террористам из Хохлостана! Даёшь им палец, а они пытаются откусить всю руку! Только у них появляется повод для так называемой «перемоги» - сразу же случается очередной жирный всплеск русофобской активности.
******************** 335 ********************
Nsubj + Verb :  мы не ни имеем, они пытаются откусить, повод появляется, всплеск сразу - случается
Appos :  
Noun+Aj (Amod) :  очередной всплеск, жирный всплеск, русофобской активности
Noun+Noun (Nmod) :  права показывать, террористам из хохлостана, повод называемой перемоги, всплеск активности
Verb + Obl/Obj :  имеем права, показывать слабину, пытаются руку откусить, откусить руку, появляется у них


In [1043]:
show_dependency(df['text'][335])

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
✔               | nsubj    | имеем           | [️Мы]               
️Мы             | appos    | ✔               | []                  
ни              | advmod   | имеем           | [в, коем, случае]   
в               | fixed    | ни              | []                  
коем            | fixed    | ни              | []                  
случае          | fixed    | ни              | []                  
не              | advmod   | имеем           | []                  
имеем           | ROOT     | имеем           | [✔, ни, не, права, !]
права           | obj      | имеем           | [показывать]        
показывать      | nmod     | права           | [слабину, террористам]
слабину         | obj      | показывать      | []                  
террористам     | iobj     | показывать      | [Хохлостана]        
из              | case     | Хохлостана   

In [177]:
df

,text,core_noun_verb,verb_obl_obj,core_noun_verb_str,appos,amod,nmod,comp,verb_obl_obj_str,appos_str,amod_str,nmod_str
0,Вагнер отправляет к бандере и прочим шайтанам ...,"[[вагнер, отправляет]]","[[отправляет, к, бандере], [отправляет, бригады]]",вагнер отправляет,"[[батальон, джохара, дудаева]]","[[прочим, шайтанам], [следующие, бригады], [ша...","[[бандере, бахмута, районе], [районе, бахмута]...",[],"отправляет к бандере, отправляет бригады",батальон джохара дудаева,"прочим шайтанам, следующие бригады, шайтанский...","бандере бахмута районе, районе бахмута, бригад..."
1,Тут понимаешь всё про метафизику этой войны. «...,"[[укропы, вышли], [они, вышли]]","[[тут, понимаешь, всё], [тут, понимаешь, про, ...","укропы вышли, они вышли",[],[],"[[метафизику, войны], [укропы, запада, покрово...",[],"тут понимаешь всё, тут понимаешь про метафизик...",,,"метафизику войны, укропы запада покровом, покр..."
2,"Наш источник сообщает, что остатки военной ави...","[[источник, сообщает, переброшена]]","[[переброшена, на, аэродром]]",источник сообщает переброшена,"[[аэродром, рог]]","[[военной, авиации], [долгинцево(кривой, рог]]","[[остатки, авиации], [остатки, укрорейха]]",[],переброшена на аэродром,аэродром рог,"военной авиации, долгинцево(кривой рог","остатки авиации, остатки укрорейха"
3,Раньше укронацисты строили песчаные замки — но...,"[[укронацисты, раньше, строили], [укронацисты,...","[[раньше, строили, замки]]","укронацисты раньше строили, укронацисты теперь...",[],"[[песчаные, замки]]",[],[],раньше строили замки,,песчаные замки,
4,Именно эти регионы являются настоящим рассадни...,"[[регионы, являются, рассадником], [регионы, и...","[[имеют, для, укронациков], [имеют, смысл]]","регионы являются рассадником, регионы имеют",[],"[[настоящим, рассадником], [сакральный, смысл]...","[[рассадником, укронацизма]]",[],"имеют для укронациков, имеют смысл",,"настоящим рассадником, сакральный смысл, идеол...",рассадником укронацизма
...,...,...,...,...,...,...,...,...,...,...,...,...
3638,Стоимость очередной подачки хохлам - $675 млн.,[],[],,[],"[[очередной, подачки], [-, млн]]","[[стоимость, подачки], [подачки, млн]]",[],,,"очередной подачки, - млн","стоимость подачки, подачки млн"
3639,Хохлячие каналы ссылаясь на некую разведку соо...,"[[каналы, сообщают], [вс, готовы, подорвать], ...","[[ссылаясь, на, разведку], [сообщают, о, том],...","каналы сообщают, вс готовы подорвать, зеленски...",[],"[[хохлячие, каналы], [каховской, гэс], [масшта...","[[каналы, ссылаясь], [дамбу, гэс], [шумок, тре...",[],"ссылаясь на разведку, сообщают о том, подорват...",,"хохлячие каналы, каховской гэс, масштабной тре...","каналы ссылаясь, дамбу гэс, шумок тревоги"
3640,"Обсуждают недоукраинца Невзорова, Анну Нетребк...",[],"[[обсуждают, недоукраинца]]",,[],[],[],[],обсуждают недоукраинца,,,
3641,И вот приходит коллективный Зусько и буквально...,"[[зусько, вот, приходит], [зусько, буквально, ...","[[буквально, начинает, хохлов, заваливать], [з...","зусько вот приходит, зусько буквально начинает...",[],"[[коллективный, зусько], [ровном, месте]]","[[трупами, мобилизованных]]",[],"буквально начинает хохлов заваливать, завалива...",,"коллективный зусько, ровном месте",трупами мобилизованных


In [178]:
df.to_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/dataset_v_collocations.csv')